In [1]:
import pyspark
import pyspark.sql.functions as F
import pyspark.sql.types as T
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [2]:
rev_df = spark.read.json("/mnt/raid0_ssd_8tb/dataset/Amazon/Category/reviews_Musical_Instruments.json")
meta_df = spark.read.json("/mnt/raid0_ssd_8tb/dataset/Amazon/Category/meta_Musical_Instruments.json")

22/09/15 21:48:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
rev_df

asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0006428320,"[0, 0]",3.0,The portfolio is ...,"03 11, 2014",A1YS9MDZP93857,John Taylor,Parts missing,1394496000
0014072149,"[0, 0]",5.0,If you are a seri...,"06 6, 2013",A3TS466QBAWB9D,Silver Pencil,Perform it with a...,1370476800
0041291905,"[0, 0]",5.0,This is and excel...,"10 14, 2013",A3BUDYITWUSIS7,joyce gabriel cor...,Vivalldi's Four S...,1381708800
0041913574,"[0, 0]",5.0,Perfect for someo...,"09 23, 2010",A19K10Z0D2NTZK,TexasCowboy,Full score: voice...,1285200000
0201891859,"[0, 1]",1.0,How many Nocturne...,"10 17, 2012",A14X336IB4JD89,dfjm53,Unable to determi...,1350432000
0577088726,"[0, 0]",5.0,this was written ...,"06 14, 2013",A2HR0IL3TC4CKL,"scarecrow ""scarec...",arduous indeed!,1371168000
0634029231,"[0, 0]",3.0,Greg Koch is a kn...,"06 24, 2005",A2DHYD72O52WS5,"Amazon Customer ""...",GREAT! BUT NOT FO...,1119571200
0634029347,"[2, 12]",2.0,I bought this DVD...,"10 15, 2005",A1MUVHT8BONL5K,"Amazon Customer ""...",NOT CLAPTION MUSI...,1129334400
0634029347,"[0, 1]",1.0,I was looking for...,"12 21, 2013",A15GZQZWKG6KZM,Robert Wright,I was disapointed,1387584000
0634029347,"[1, 8]",5.0,If you are an una...,"10 2, 2005",A16WE7UU0QD33D,S. R. Mcdonald,Solo Years Revisited,1128211200


In [4]:
rev_df.count()

500176

In [288]:
rev_df.select(F.count_distinct("reviewerID"))

count(DISTINCT reviewerID)
339231


In [5]:
equals_zero = lambda e: e == 0
rev_df = rev_df.withColumn("drop", F.exists(F.col("helpful"), equals_zero))
rev_df = rev_df.filter(F.col("drop") == False).drop(F.col("drop"))
rev_df = rev_df.select("asin", "overall", "reviewText", "reviewerID", "reviewerName", "summary", rev_df.helpful[0], rev_df.helpful[1], "unixReviewTime", "reviewTime")
rev_df

asin,overall,reviewText,reviewerID,reviewerName,summary,helpful[0],helpful[1],unixReviewTime,reviewTime
0634029347,2.0,I bought this DVD...,A1MUVHT8BONL5K,"Amazon Customer ""...",NOT CLAPTION MUSI...,2,12,1129334400,"10 15, 2005"
0634029347,5.0,If you are an una...,A16WE7UU0QD33D,S. R. Mcdonald,Solo Years Revisited,1,8,1128211200,"10 2, 2005"
0634029355,2.0,I was carefull to...,AXMWZYP2IROMP,Alex,not great instruc...,2,2,1352419200,"11 9, 2012"
0634029355,2.0,I bought this DVD...,A1MUVHT8BONL5K,"Amazon Customer ""...",NOT CLAPTION MUSI...,3,17,1129334400,"10 15, 2005"
0634029355,5.0,Unless you are so...,A6DCKXX4659CR,"Hotshot ""Hotshot""",CLAPTON GUITAR IN...,2,2,1189728000,"09 14, 2007"
0634029355,5.0,I've played guita...,A28YJZCV43ZWQW,James V. Signorile,First rate lesson...,3,3,1206489600,"03 26, 2008"
0634029355,2.0,This isn't worth ...,A2I4CV4PGZCNAF,Josh Hayden,Don't waste your ...,2,3,1271721600,"04 20, 2010"
0634029363,2.0,As much as I like...,A1M7OWNI93N1Y3,geronimo,not very didactic,1,1,1365897600,"04 14, 2013"
0634029363,1.0,THIS IS NOT A CLA...,A2FEDDUHQAEN97,"J. Drinks ""chesap...",clapton acoustic,12,31,1117497600,"05 31, 2005"
0634029363,5.0,For those who may...,A338HVUM4AWJY7,Joaquim Do M. Nt ...,This is an instru...,8,8,1182038400,"06 17, 2007"


In [6]:
rev_df = rev_df.groupBy("reviewerID").agg(F.sum("helpful[0]").alias("helpful_0"), \
                                F.sum("helpful[1]").alias("helpful_1"), \
                                F.collect_list("asin").alias("asin"), \
                                F.collect_list("overall").alias("overall"), \
                                F.collect_list("reviewText").alias("reviewText"), \
                                F.collect_list("summary").alias("summary"), \
                                F.collect_list("reviewerName").alias("reviewerName"), \
                                F.collect_list("unixReviewTime").alias("unixReviewTime"), \
                                F.collect_list("reviewTime").alias("reviewTime")
                                )

In [7]:
rev_df

reviewerID,helpful_0,helpful_1,asin,overall,reviewText,summary,reviewerName,unixReviewTime,reviewTime
A0099919A30Q9445KCR1,2,3,[B0014ZG676],[5.0],[This was bought ...,[Good recorder],"[FScott Ferrell ""...",[1355184000],"[12 11, 2012]"
A03177322D67XWBT5...,3,3,"[B000J5Y282, B005...","[5.0, 5.0]",[Awesome product....,[More than I expe...,"[Fernando, Fernando]","[1363564800, 1363...","[03 18, 2013, 03 ..."
A0400898QDJDPR9X97QW,1,2,[B0002DUPZK],[5.0],[I can't get out ...,[New Experiance !!!],[verna garrison],[1353715200],"[11 24, 2012]"
A0602489S9C6EVXEOJI1,1,2,[B007V7R2KG],[1.0],[Second one I rec...,[Headphones],[Pen Name],[1371600000],"[06 19, 2013]"
A07757242U01IL7PU...,1,1,[B0087UPZEG],[5.0],"[At a low cost, t...",[Great buy and wo...,[Omni Explorer],[1364169600],"[03 25, 2013]"
A08161909WK3HU7UYTMW,1,1,[B000A39FMQ],[5.0],[I WAS GROWING UP...,[HAY I LIKE THE S...,[Jody G. Collyer ...,[1376006400],"[08 9, 2013]"
A089543336B2HF35W...,1,2,[B000ULAP4U],[2.0],[The tone product...,[nope*],[Jun Lin],[1386806400],"[12 12, 2013]"
A100HNWBUEX9NF,1,1,[B000SKO0OY],[5.0],[The whole Paul P...,[Paul Potts],[R. Gibbens Robic...,[1196467200],"[12 1, 2007]"
A100TCEB7KB4B,2,3,[B003HLEF7I],[5.0],"[I agree, the SP-...",[SP-55],[Thomas Luke Andr...,[1316563200],"[09 21, 2011]"
A100US6QVOOXLX,1,1,[B000AAGM0M],[4.0],[I purchased this...,[Good product],"[R. Lycan ""Tuba R...",[1197936000],"[12 18, 2007]"


In [8]:
rev_df.count()

158059

In [9]:
rev_df = rev_df.withColumn("label", (F.col("helpful_0") / F.col("helpful_1"))).drop("helpful_0").drop("helpful_1")
rev_df = rev_df.filter((F.col("label") > 0.8) | (F.col("label") < 0.2))
rev_df = rev_df.withColumn("label", F.udf(lambda a: 1 if a > 0.8 else 0, T.IntegerType())(F.col("label")))

In [10]:
rev_df

reviewerID,asin,overall,reviewText,summary,reviewerName,unixReviewTime,reviewTime,label
A03177322D67XWBT5...,"[B000J5Y282, B005...","[5.0, 5.0]",[Awesome product....,[More than I expe...,"[Fernando, Fernando]","[1363564800, 1363...","[03 18, 2013, 03 ...",1
A07757242U01IL7PU...,[B0087UPZEG],[5.0],"[At a low cost, t...",[Great buy and wo...,[Omni Explorer],[1364169600],"[03 25, 2013]",1
A08161909WK3HU7UYTMW,[B000A39FMQ],[5.0],[I WAS GROWING UP...,[HAY I LIKE THE S...,[Jody G. Collyer ...,[1376006400],"[08 9, 2013]",1
A100HNWBUEX9NF,[B000SKO0OY],[5.0],[The whole Paul P...,[Paul Potts],[R. Gibbens Robic...,[1196467200],"[12 1, 2007]",1
A100US6QVOOXLX,[B000AAGM0M],[4.0],[I purchased this...,[Good product],"[R. Lycan ""Tuba R...",[1197936000],"[12 18, 2007]",1
A1016EFJZN9WZU,[B0002GLMEK],[5.0],[Love this uke. S...,[Awesome uke for ...,[AJEDrew7],[1315008000],"[09 3, 2011]",1
A101OE0F4EO2AE,[B003BVJ7Q8],[4.0],[Labeling cables ...,[Great product],[Lowell Thomas],[1363046400],"[03 12, 2013]",1
A102LR8KJE6UZZ,[B000OMG0KI],[5.0],[I've tried many ...,[best humidifer s...,[david butterworth],[1384300800],"[11 13, 2013]",1
A102MK78XH84JN,[B0002DV7ZM],[5.0],[I got very tired...,[I love this thing],"[Jason B ""Indeed""]",[1371081600],"[06 13, 2013]",1
A103ESFXL63GO7,[B0018TJ602],[5.0],[This is a very n...,[Tenor Uke],[teddyzbabe],[1309305600],"[06 29, 2011]",1


In [11]:
rev_df = rev_df.withColumn("reviewLen", F.udf(lambda a: len(a), T.IntegerType())(F.col("reviewerName")))
rev_df = rev_df.withColumn("reviewSetLen", F.udf(lambda a: len(set(a)), T.IntegerType())(F.col("reviewerName")))
rev_df = rev_df.drop(F.col("reviewLen")).drop(F.col("reviewSetLen"))

In [12]:
# rev_df_json = rev_df.toJSON()
rev_df.coalesce(1).write.format('json').save("/mnt/raid0_ssd_8tb/dataset/Amazon/Refined/Musical_Instruments")

In [13]:
rev_df.printSchema()

root
 |-- reviewerID: string (nullable = true)
 |-- asin: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- overall: array (nullable = false)
 |    |-- element: double (containsNull = false)
 |-- reviewText: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- summary: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- reviewerName: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- unixReviewTime: array (nullable = false)
 |    |-- element: long (containsNull = false)
 |-- reviewTime: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- label: integer (nullable = true)



In [107]:
# id_list = list(rev_df.select('reviewerID').toPandas()['reviewerID'])

In [108]:
# s = set()
# dup = list()
# for r_id in id_list:
#     if r_id in s:
#         dup.append(r_id)
#     else:
#         s.add(r_id)

In [109]:
# print(dup)

['A1MUVHT8BONL5K', 'A1R8OFZY6CYX7G', 'A15STV0U0MEAS0', 'A2OT7AEHVZ0AZ0', 'A3V5O5G3I1T2NX', 'A18BD98SU49UB7', 'A3Q1FVRV0CRYTH', 'A3R2YB0WTTB0IJ', 'AW4MT9WP95TVD', 'A1R8OFZY6CYX7G', 'AM2PV53UX1JEQ', 'A2K3SULCNHLSQ5', 'A16QODENBJVUI1', 'A14CWMXH8UTAFF', 'A9Q28YTLYREO7', 'A2INX8B6VXUVW6', 'AQ4B80F0JSBNI', 'A9Q28YTLYREO7', 'A2AIMXT9PLAM12', 'A2VE83MZF98ITY', 'AYOPOFV2QKC31', 'AYOPOFV2QKC31', 'AUEHG0DB54B7K', 'A33GGROUQRQZS', 'A33GGROUQRQZS', 'AUEHG0DB54B7K', 'A33GGROUQRQZS', 'AQ4B80F0JSBNI', 'A3CN9CCJUNIPKT', 'A2BL89A0TI4ASS', 'A3R01M8PZ4XJ1N', 'AUHYM80NBD8MC', 'A2TC5QDBYCJITD', 'AQ4B80F0JSBNI', 'A1UQ1BU51CDB53', 'A1SK0PB9ORI6XZ', 'A3R01M8PZ4XJ1N', 'A3R01M8PZ4XJ1N', 'AZRW3N2G7B185', 'AZRW3N2G7B185', 'AYPCUQS6ARWFH', 'A3R01M8PZ4XJ1N', 'A1I4HJDNBC0CF3', 'A2VE83MZF98ITY', 'A16CZRQL23NOIW', 'A2NYK9KWFMJV4Y', 'A2JWPVOP3SRHEN', 'A1UQ1BU51CDB53', 'A2AIMXT9PLAM12', 'A70WPHE33U5B', 'A1F89OE7BAM7EL', 'A2AIMXT9PLAM12', 'A2AIMXT9PLAM12', 'ALAJL3S09HBS7', 'A328S9RN3U5M68', 'A70WPHE33U5B', 'AYOPOFV2QKC31

In [110]:
# rev_df.filter(rev_df.reviewerID == 'A1R8OFZY6CYX7G').show()

+----------+-------+--------------------+--------------+----------------+--------------------+----------+----------+
|      asin|overall|          reviewText|    reviewerID|    reviewerName|             summary|helpful[0]|helpful[1]|
+----------+-------+--------------------+--------------+----------------+--------------------+----------+----------+
|0739040375|    2.0|Part 1 of this DV...|A1R8OFZY6CYX7G|Howard L Lambert|entertaining, but...|         3|         4|
|0739048287|    5.0|I own 12 other gu...|A1R8OFZY6CYX7G|Howard L Lambert|in my top 3 favor...|         5|         5|
|1417030321|    3.0|I'm not sure why ...|A1R8OFZY6CYX7G|Howard L Lambert| not a 5 star movie!|         9|        56|
|B0002GLCRC|    4.0|The instructions ...|A1R8OFZY6CYX7G|Howard L Lambert|doesn't look like...|         5|         7|
+----------+-------+--------------------+--------------+----------------+--------------------+----------+----------+



In [65]:
rev_df.select(count_distinct("reviewerID")).show()

+--------------------------+
|count(DISTINCT reviewerID)|
+--------------------------+
|                    158059|
+--------------------------+



In [8]:
rev_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)
 |-- drop: boolean (nullable = true)



In [9]:
meta_df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- imUrl: string (nullable = true)
 |-- price: double (nullable = true)
 |-- related: struct (nullable = true)
 |    |-- also_bought: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- also_viewed: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- bought_together: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- buy_after_viewing: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- salesRank: struct (nullable = true)
 |    |-- Arts, Crafts & Sewing: long (nullable = true)
 |    |-- Beauty: long (nullable = true)
 |    |-- Books: long (nullabl

In [221]:
meta_df

_corrupt_record,asin,brand,categories,description,imUrl,price,related,salesRank,title
null,0006428320,null,[[Musical Instrum...,null,http://ecx.images...,17.95,null,"{null, null, null...",Six Sonatas For T...
null,0014072149,,[[Musical Instrum...,Composer: J.S. Ba...,http://ecx.images...,18.77,"{null, [B0058DK7R...","{null, null, null...",Double Concerto i...
null,0041291905,null,[[Musical Instrum...,Vivaldi's famous ...,http://ecx.images...,null,null,"{null, null, null...",Hal Leonard Vival...
null,0041913574,null,[[Musical Instrum...,444 pages. \nRepr...,http://ecx.images...,49.99,null,null,Aida: Opera in Qu...
null,0201891859,null,[[Musical Instrum...,null,http://ecx.images...,null,null,"{null, null, null...",Nocturnes
null,0577088726,null,[[Musical Instrum...,Sisyphus Redux fo...,http://g-ecx.imag...,null,null,null,Sisyphus Redux fo...
null,0634029363,null,"[[CDs & Vinyl, Sp...",Learn to play eig...,http://ecx.images...,18.99,"{[0634029355, 063...","{null, null, null...",Hal Leonard Eric ...
null,0634029355,null,"[[CDs & Vinyl, Sp...",Learn the riffs a...,http://ecx.images...,13.48,"{[0634029347, 063...","{null, null, null...",Hal Leonard Eric ...
null,0634029347,null,"[[CDs & Vinyl, Sp...",Riffs and solos f...,http://ecx.images...,16.19,"{[0634029355, 063...","{null, null, null...",Hal Leonard Eric ...
null,0634029231,null,"[[Movies & TV, Mo...",null,http://ecx.images...,null,null,"{null, null, null...",Best of Stevie Ra...


In [11]:
meta_df.count()

84901